In [1]:
import numpy as np
import pandas as pd

import pickle

In [40]:
df = pd.read_pickle('../Data/01_sf_recommender_pca')

In [3]:
df.head()

,company_name,job_title,listed_items,posting_url,0,1,2,3,4,5,...,90,91,92,93,94,95,96,97,98,99
0,Gap Inc. Corporate,"Software Engineer, Price Execution",write build product according business need co...,https://www.indeed.com/rc/clk?jk=77d524a7cf198...,-0.019090,0.040501,-0.112431,0.103970,-0.106989,-0.026306,...,0.002396,-0.015071,-0.013495,0.002928,-0.015414,-0.079520,0.011122,0.001284,0.034969,0.043990
1,WrkShp,Business Analyst,closely product assist investigation deep dive...,https://www.indeed.com/company/WrkShp/jobs/Bus...,-0.010927,0.164458,0.122718,-0.153918,-0.014575,-0.100430,...,-0.043133,-0.011965,0.003011,0.034008,0.033314,0.005770,-0.075077,0.064688,-0.001266,-0.017888
2,Ceres Imaging,Image Processing: GIS / Remote Sensing Analyst,proficiency gi software e g arcgis envi experi...,https://www.indeed.com/rc/clk?jk=8f702cd563785...,0.064959,-0.130597,-0.012915,0.000326,-0.020176,-0.031159,...,0.072121,0.020364,-0.057734,-0.022652,0.023028,-0.026972,0.045027,0.024582,0.061548,-0.015834
3,Deloitte,"Analyst, Strategy and Research",effectively interpret client request use tacti...,https://www.indeed.com/rc/clk?jk=8a288a5c5a09d...,-0.101598,-0.037274,0.040113,-0.012223,-0.040209,0.047943,...,-0.014547,0.000741,0.046328,0.090302,0.034393,-0.004047,-0.007654,-0.034240,0.002480,-0.023435
4,Turing Video,Computer Vision Software Engineer,maintain existing software system implement co...,https://www.indeed.com/rc/clk?jk=fcf308f2fee2a...,0.149299,0.010717,-0.028881,0.143432,-0.067979,0.051231,...,-0.041999,0.037092,-0.027940,-0.001335,-0.017423,0.007670,-0.051687,-0.014250,-0.029634,-0.004588


In [5]:
def make_recommendation(df, job_index,n_recommendations=5):
    n_recommendations += 1
    df['similarity'] = df.iloc[:,4:].T.apply(lambda x: np.dot(x,df.iloc[job_index,4:]))
    largest_indeces = df.similarity.nlargest(n_recommendations).index
    largest_indeces = largest_indeces[1:]
    recommendations = df.iloc[largest_indeces].posting_url
    original_posting = df.iloc[job_index].posting_url
    return recommendations, original_posting

In [6]:
recommendations, original_posting = make_recommendation(df, 2479,n_recommendations=5)

In [8]:
original_posting

'https://www.indeed.com/rc/clk?jk=61c67daed15efa5f&fccid=330c28c156c5fb55&vjs=3'

In [7]:
recommendations.values

array(['https://www.indeed.com/rc/clk?jk=d0a7e4f18722fc85&fccid=fd86291e008a4edf&vjs=3',
       'https://www.indeed.com/rc/clk?jk=de4582911b6e7615&fccid=734e88fca63788fc&vjs=3',
       'https://www.indeed.com/rc/clk?jk=fda80b56a3907f5d&fccid=c2aed722996a4145&vjs=3',
       'https://www.indeed.com/rc/clk?jk=4e3e375496da68e2&fccid=7c3a1f1f98dde031&vjs=3',
       'https://www.indeed.com/rc/clk?jk=8ee64f8287f7919a&fccid=a4e4e2eaf26690c9&vjs=3'],
      dtype=object)

In [30]:
df.iloc[2479].posting_url

'https://www.indeed.com/rc/clk?jk=61c67daed15efa5f&fccid=330c28c156c5fb55&vjs=3'

In [29]:
df.shape

(2964, 105)

## Turn Text Description into a Vector

In [4]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
job_desc = 'Research and experiment with different machine learning algorithms and techniques for item classification, product recognition and matching, attribute extraction. \
            Conduct design and code reviews. \
            Productionize the developed Machine Learning solutions. \
            Work with engineers to make sure the engines scale well on high volumes of data. \
            MS/PhD in Computer Science or a related field. \
            3+ years of industry experience in data science and machine learning preferred. \
            Expertise in Machine Learning. \
            Strong CS fundamentals, such as algorithms and data structures. \
            Expertise in Python. \
            Proficiency with relational databases such as MySQL. \
            Experience with cloud computing stacks such as Amazon Web Services preferred. \
            Excellent written and verbal communication skills. \
            Enthusiasm for working hard and having fun in a dynamic environment.'
        


In [6]:
def format_text(text):
    
    text = re.sub('[\W_]+', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text.lower())
    
    stop = set(stopwords.words('english'))
    text = [item for item in text.split() if item not in stop]
    
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(token) for token in text]
    
    stop2 = ['data', 'code', 'team', 'work']
    text = [item for item in text if item not in stop2]
    
    return text

In [7]:
job_desc = format_text(job_desc)

In [8]:
def create_ngrams(text):
    
    doc_ngrams = []
    
    for i, j in zip(text, text[1:]):
        bigram = '_'.join([i, j])
        doc_ngrams.append(bigram)
        
    for i, j, k in zip(text, text[1:], text[2:]):
        trigram = '_'.join([i, j, k])
        doc_ngrams.append(trigram)
        
    for i, j, k, l in zip(text, text[1:],
                          text[2:], text[3:]):
        quadgram = '_'.join([i, j, k, l])
        doc_ngrams.append(quadgram)
        
    pickle_in = open('../Tools_and_models/top_ngrams','rb')
    top_ngrams = pickle.load(pickle_in)
    pickle_in.close()
        
    top_ngrams = top_ngrams.split()
    
    for doc_ngram in doc_ngrams:
        for top_ngram in top_ngrams:
            if doc_ngram == top_ngram:
                text.append(doc_ngram)
            else:
                pass
    return ' '.join(text)
    

In [9]:
job_desc = create_ngrams(job_desc)

In [10]:
def vectorize(text):
    pickle_in = open('../Tools_and_models/tf_idf_vectorizer','rb')
    tf_idf_vectorizer = pickle.load(pickle_in)
    pickle_in.close()
    
    pickle_in = open('../Tools_and_models/tf_idf_model','rb')
    tf_idf = pickle.load(pickle_in)
    pickle_in.close()
    
    text_vector_array = tf_idf_vectorizer.transform([text]).toarray()
    
    text_vector = pd.DataFrame(text_vector_array,columns=tf_idf.get_feature_names())
    
    return text_vector
    

In [11]:
text_vector = vectorize(job_desc)
text_vector

,00,000,000ad_hoc,000best_practice,000machine_learning,000skill_experience,007ability_independently,007end_end,00am,00pm,...,zookeeperend_end,zoom,zoura,zuckerberg,zurb,zvs,zweigwhite,zymergen,zymo,zynga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
def pcaitize(text_vector):

    pickle_in = open('../Tools_and_models/pca_tool','rb')
    pca = pickle.load(pickle_in)
    pickle_in.close()
    
    text_pca = pca.transform(text_vector)
    
    return pd.DataFrame(text_pca)

In [36]:
text_pca = pcaitize(text_vector)
text_pca

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.313629,0.059027,0.188926,0.027239,0.1554,0.155671,-0.064135,0.011923,0.018943,-0.059297,...,-0.00635,-0.006189,-0.030233,0.013168,-0.021185,0.020178,-0.037472,-0.015292,-0.001556,0.022535


## Making a search based on our text_pca

In [37]:
def make_recommendation(df, text_pca,n_recommendations=5):
    n_recommendations += 1
    df['similarity'] = df.iloc[:,4:].T.apply(lambda x: np.dot(x,text_pca.iloc[0,:]))
    largest_indeces = df.similarity.nlargest(n_recommendations).index
    recommendations = df.iloc[largest_indeces].posting_url
    return recommendations

In [41]:
recommendations = make_recommendation(df, text_pca,5)
recommendations.values

array(['https://www.indeed.com/rc/clk?jk=795dfed0082171ee&fccid=250b3b23ab6d1a65&vjs=3',
       'https://www.indeed.com/rc/clk?jk=0ffe555ff7c0bcb2&fccid=b62884b388b48696&vjs=3',
       'https://www.indeed.com/rc/clk?jk=25dea4c8feffdcfe&fccid=037150182eaec9aa&vjs=3',
       'https://www.indeed.com/rc/clk?jk=dd0fc449ef317b70&fccid=e78beff0619871a0&vjs=3',
       'https://www.indeed.com/rc/clk?jk=c8d268fd9541ce48&fccid=4444adbe1bdb3f80&vjs=3',
       'https://www.indeed.com/rc/clk?jk=c2e0edc807d1768b&fccid=e07842a8df6e60cc&vjs=3'],
      dtype=object)